In [1]:
from __future__ import division
import os
import sys

from datetime import datetime
from functools import partial
import re
import time

import numpy as np
import pandas as pd

from Bio import Entrez
Entrez.email = "patel.ravip@temple.edu"

from IPython.display import display
from ipywidgets import *

In [2]:
genePattern = re.compile(r"""GENE=([A-Z0-9,:\-]+)""")

In [3]:
df = pd.read_csv('/DATA/raw/dbpshp/unique_snps_rsids.txt', header=None, names=['chrom','chrom_pos','rsid'], sep='\t')

In [4]:
rsid = 'rs78370858'

In [5]:
def get_gene(row):
    time.sleep(0.34)
    rsid = row['rsid']
    rs = rsid.strip('rs')
    h = Entrez.esummary(db='SNP', id=rs)
    r = Entrez.read(h)
    h.close()
    
    try:
        data = r[0]
    except KeyboardInterrupt:
        raise
    except:
        return (row['chrom'],row['chrom_pos'],row['rsid'],np.nan,np.nan)       
        
    fx = data['FXN_CLASS']
    summary = data['DOCSUM']
    res = genePattern.search(summary)
    try:
        entries = res.groups()[0].split(",")
        genes = ",".join([i.split(":")[0] for i in entries])
    except KeyboardInterrupt:
        raise
    except:
        return (row['chrom'],row['chrom_pos'],row['rsid'],np.nan,np.nan)
    else:
        return (row['chrom'],row['chrom_pos'],row['rsid'],genes,fx)

In [6]:
%%time
pb = IntProgress(width='800px')
pb.value = 0
pb.max = df.shape[0]
ph = HTML('{} / {}'.format(pb.value,pb.max))
display(pb,ph)

print 'Starting: {}'.format(datetime.now().strftime('%b %d, %Y %H:%M:%S'))

with open('/DATA/raw/dbpshp/snp_annot.txt', 'w') as fh:
    for ix,row in df.iterrows():
        try:
            res = get_gene(row)
            fh.write('{}\n'.format('\t'.join(map(str,res))))
        except KeyboardInterrupt:
            raise
        except:
            continue
        finally:
            pb.value += 1
            ph.value = '{} / {}'.format(pb.value, pb.max)
            
print 'Done: {}'.format(datetime.now().strftime('%b %d, %Y %H:%M:%S'))

Starting: Apr 10, 2017 16:36:29
Done: Apr 10, 2017 19:29:27
CPU times: user 2min 7s, sys: 20.8 s, total: 2min 28s
Wall time: 2h 52min 58s


In [7]:
df.head()

,chrom,chrom_pos,rsid
0,1,864938,rs78370858
1,1,1140435,rs1815606
2,1,1209636,rs3737722
3,1,1210471,rs11260577
4,1,1211292,rs6685064


In [8]:
df.ix[10146:10148]

,chrom,chrom_pos,rsid
10146,5,119683684,rs10058597
10147,5,119686188,rs6863052
10148,5,119697716,rs6896794
